In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install --upgrade sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=1ebdddf231cb5038bea50cd06c86afe337d4b4e8865abf3708e394acda9321c8
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torch.utils.checkpoint import checkpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score

import gluonnlp as nlp
import sentencepiece
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel
from transformers.optimization import get_cosine_schedule_with_warmup

from tqdm import tqdm
import json

# Device 설정(GPU 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


# KoBERT Sentiment Analysis

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BERT_modeling/labeled_dataset.csv')
df

,Unnamed: 0.1,Unnamed: 0,id,rank,company,name1,name2,category,volume,alc,...,score,real_date,buy,real_review,split_sentences,sentence,normalized,words,polarity,sentiment
0,0,0,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",상한 겁니까,상한 겁니까,상한 걸,-3.0,negative
1,1,1,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",구매확정합니다,구매확정합니다,구매 확정,0.0,neutral
2,2,2,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,올바르 상품 오 그 먹,6.0,positive
3,3,3,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상태,올바른 상태,올바르 상태,3.0,positive
4,4,4,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,저번 시키 박스 제품 상하 뒤집 말 스티커 올바르 박스,3.0,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302731,305307,305633,147662,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,12.0,...,5,22.08.21.,우리술한잔,맛있네요 택배 포장 걱정했는데\n너무 만족으럽게 포장해 주셔서 잘 받았습니다.\n다...,"['맛있네요', '택배 포장 걱정했는데\n너무 만족으럽게 포장해 주셔서 잘 받았습니...",맛있네요,맛있네요,맛있,3.0,positive
302732,305309,305635,147663,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,12.0,...,5,22.06.07.,술마켓,꼼콤하게 잘 왔구요 걸쭉하니 만납니다.,"['꼼콤하게 잘 왔구요', '걸쭉하니 만납니다.']",꼼콤하게 잘 왔고요,꼼콤하게 잘 왔고요,꼬 오,0.0,neutral
302733,305310,305636,147663,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,12.0,...,5,22.06.07.,술마켓,꼼콤하게 잘 왔구요 걸쭉하니 만납니다.,"['꼼콤하게 잘 왔구요', '걸쭉하니 만납니다.']",걸쭉하니 만납니다,걸쭉하니 만납니다,만나,-1.5,negative
302734,305311,305637,147664,121,주로,골목막걸리 프리미엄 12,골목막걸리 프리미엄 12,막걸리/탁주,350,12.0,...,5,22.07.06.,술마켓,따봉 맛있어요 얼음 넣으면 신의한수,"['따봉 맛있어요', '얼음 넣으면 신의한수']",따봉 맛있어요,따봉 맛있어요,따 봉 맛있,3.0,positive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302736 entries, 0 to 302735
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0.1     302736 non-null  int64  
 1   Unnamed: 0       302736 non-null  int64  
 2   id               302736 non-null  int64  
 3   rank             302736 non-null  int64  
 4   company          302736 non-null  object 
 5   name1            302736 non-null  object 
 6   name2            302736 non-null  object 
 7   category         302736 non-null  object 
 8   volume           302736 non-null  int64  
 9   alc              302736 non-null  float64
 10  type             302736 non-null  object 
 11  price            302736 non-null  int64  
 12  review_num       302736 non-null  int64  
 13  click            302736 non-null  int64  
 14  link             302736 non-null  object 
 15  score            302736 non-null  int64  
 16  real_date        302736 non-null  obje

In [ ]:
# 품사태깅 이뤄지지 않은 결측값 drop
df = df.dropna(subset=['words'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302736 entries, 0 to 302735
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0.1     302736 non-null  int64  
 1   Unnamed: 0       302736 non-null  int64  
 2   id               302736 non-null  int64  
 3   rank             302736 non-null  int64  
 4   company          302736 non-null  object 
 5   name1            302736 non-null  object 
 6   name2            302736 non-null  object 
 7   category         302736 non-null  object 
 8   volume           302736 non-null  int64  
 9   alc              302736 non-null  float64
 10  type             302736 non-null  object 
 11  price            302736 non-null  int64  
 12  review_num       302736 non-null  int64  
 13  click            302736 non-null  int64  
 14  link             302736 non-null  object 
 15  score            302736 non-null  int64  
 16  real_date        302736 non-null  obje

### 1) 데이터셋 준비

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.float32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return {'input_ids': self.sentences[i][0],
        'attention_mask': self.sentences[i][1],
        'labels': self.labels[i]
        }

    def __len__(self):
        return (len(self.labels))

# Pretrained BERT 모델과 Tokenizer 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bert_model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


### 2) 모델 훈련

In [ ]:
# 하이퍼파라미터 설정
max_len = 64
batch_size = 32
max_seq_length = 128
num_epochs = 3
num_hidden_layers = 2
hidden_size=512
learning_rate = 2e-5
num_features = bert_model.config.hidden_size

In [ ]:
# 학습을 수행하는 Transformer 모델 생성(BERTDropout 적용)
class BERTRegressor(nn.Module):
    def __init__(self, bert_model, num_hidden_layers=2, hidden_size=512):
        super(BERTRegressor, self).__init__()
        self.bert_model = bert_model

        # Additional hidden layers
        layers = []
        for _ in range(num_hidden_layers):
            layers.append(nn.Linear(bert_model.config.hidden_size, hidden_size))
            layers.append(nn.Dropout(0.1))
            bert_model.config.hidden_size = hidden_size

        self.hidden_layers = nn.Sequential(*layers)

        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(bert_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]  # Access the last hidden state from the tuple
        cls_output = last_hidden_state[:, 0, :]  # Get [CLS] token's output from the last hidden state

        # Apply additional hidden layers
        cls_output = self.hidden_layers(cls_output)

        cls_output = self.dropout(cls_output)
        logits = self.regressor(cls_output)
        return logits

In [ ]:
data = df[['words', 'polarity']]

# Train-Valid-Test 데이터셋 분리(7:1.5:1.5 비율)
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)
test_df, valid_df = train_test_split(test_df, test_size=0.5, random_state=42)

print('Train dataset size:', len(train_df), 'Validation dataset size:', len(valid_df), 'Test dataset size:', len(test_df))

Train dataset size: 211915 Validation dataset size: 45411 Test dataset size: 45410


In [ ]:
# Validation DataLoader with a smaller batch size
valid_batch_size = 16

data_train = BERTDataset(train_df.values, 0, 1, tokenizer, vocab, max_len, True, False)
data_valid = BERTDataset(valid_df.values, 0, 1, tokenizer, vocab,  max_len, True, False)
data_test = BERTDataset(test_df.values, 0, 1, tokenizer, vocab,  max_len, True, False)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
valid_loader = torch.utils.data.DataLoader(data_valid, batch_size=valid_batch_size, num_workers=2)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

print('Train dataset size:', len(train_loader))
print('Validation dataset size:', len(valid_loader))
print('Test dataset size:', len(test_loader))

Train dataset size: 6623
Validation dataset size: 2839
Test dataset size: 1420


In [ ]:
# Define the BERT regressor model
model = BERTRegressor(bert_model)
model = model.to(device)

# Set up the optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Number of epochs between validation checks
validation_frequency = 2

# Early stopping variables
best_validation_r2 = -float('inf')
no_improvement_count = 0
patience = 2  # Number of epochs with no improvement before early stopping

def adjusted_r2_score(r2, n_samples, n_features):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Train the model
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_r2 = 0
    total_samples = 0

    # Use tqdm to create a progress bar
    train_loader = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for batch in train_loader:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = (input_ids != vocab[vocab.padding_token]).float().to(device)
        labels = batch['labels'].to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(input_ids, attention_mask=attention_mask)
        loss = criterion(logits.squeeze(), labels)

        # Apply L2 weight regularization
        l2_lambda = 0.01
        l2_reg = torch.tensor(0.).to(device)
        for param in model.parameters():
            l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg

        total_loss += loss.item()

        # Calculate R2 score for the current batch
        predicted_labels = logits.squeeze().detach().cpu().numpy()
        ground_truth_labels = labels.detach().cpu().numpy()
        r2 = r2_score(ground_truth_labels, predicted_labels)
        total_r2 += r2

        total_samples += len(labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Update the progress bar description with the current loss and R2 score
        train_loader.set_postfix({'loss': loss.item(), 'R2': r2})

    avg_loss = total_loss / len(train_loader)
    avg_r2 = total_r2 / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss}, Average R2 Score: {avg_r2}")

    # Perform validation every few epochs
    if (epoch + 1) % validation_frequency == 0:
        model.eval()
        total_r2_validation = 0
        total_validation_samples = 0

        with torch.no_grad():
            for batch in valid_loader:
                # Validation code as before
                # Calculate R2 score for the current batch
                input_ids = batch['input_ids'].to(device)
                attention_mask = (input_ids != vocab[vocab.padding_token]).float().to(device)
                labels = batch['labels'].to(device)

                logits = model(input_ids, attention_mask=attention_mask)
                predicted_labels_validation = logits.squeeze().detach().cpu().numpy()
                ground_truth_labels_validation = labels.detach().cpu().numpy()
                r2_validation = r2_score(ground_truth_labels_validation, predicted_labels_validation)
                total_r2_validation += r2_validation

                total_validation_samples += len(labels)

        avg_r2_validation = total_r2_validation / len(valid_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation R2 Score: {avg_r2_validation}")

        # Calculate Adjusted R2 scores
        adjusted_r2_train = adjusted_r2_score(avg_r2, total_samples, num_features)  # num_features는 입력 피처의 개수입니다.
        adjusted_r2_validation = adjusted_r2_score(avg_r2_validation, total_validation_samples, num_features)

        print(f"Epoch {epoch + 1}/{num_epochs}, Average R2 Score: {avg_r2}, Adjusted R2 Score (Train): {adjusted_r2_train}")
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation R2 Score: {avg_r2_validation}, Adjusted R2 Score (Validation): {adjusted_r2_validation}")

        # Check for early stopping
        if avg_r2_validation > best_validation_r2:
            best_validation_r2 = avg_r2_validation
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        if no_improvement_count >= patience:
            print("Early stopping! Validation performance did not improve for {} epochs.".format(patience))
            break

Epoch 1/3, Average Loss: 48.06200416365653, Average R2 Score: 0.5582203827292396


Epoch 2/3, Average Loss: 42.826300874894685, Average R2 Score: 0.6418712044222687


Epoch 2/3, Validation R2 Score: 0.44307752657066196


Epoch 3/3, Average Loss: 39.10698807255732, Average R2 Score: 0.6687531608720936


In [ ]:
# Save the trained model after each epoch
torch.save(model.state_dict(), '/content/drive/MyDrive/BERT_modeling/trained_model.pth')

## 3) 모델 성능 평가

In [ ]:
# 저장된 모델 load
model = BERTRegressor(bert_model)
model = model.to(device)

# Load the saved model state dictionary
model_path = '/content/drive/MyDrive/BERT_modeling/trained_model.pth'

# Load the trained weights and state of the optimizer from the saved file
checkpoint = torch.load(model_path)

new_state_dict = {}
for key in checkpoint.keys():
    new_key = key.replace("bert_model.", "")  # Remove the prefix "bert_model."
    new_state_dict[new_key] = checkpoint[key]

# Load the filtered model's state_dict
model.load_state_dict(new_state_dict, strict=False)  # Use strict=False to ignore missing keys

_IncompatibleKeys(missing_keys=['bert_model.embeddings.word_embeddings.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.self.query.weight', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.0.attention.self.key.weight', 'bert_model.encoder.layer.0.attention.self.key.bias', 'bert_model.encoder.layer.0.attention.self.value.weight', 'bert_model.encoder.layer.0.attention.self.value.bias', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.intermediate.dense.weight', 'bert_model.encoder.layer.0.intermediate.dense.bias', 'bert_model.encoder.layer.0.output.dense

In [ ]:
# Evaluate the model on the test set
model.eval()
predictions = []
ground_truth_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = (input_ids != vocab[vocab.padding_token]).float().to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        logits = model(input_ids, attention_mask=attention_mask)

        # Collect predictions and ground truth labels
        predictions.extend(logits.squeeze().cpu().numpy())
        ground_truth_labels.extend(labels.cpu().numpy())

# Calculate R2 score on the test set
r2_test = r2_score(ground_truth_labels, predictions)

# Calculate the number of input features (sequence length)
num_features = test_loader.dataset[0]['input_ids'].size(0)

# Calculate Adjusted R2 score on the test set
adjusted_r2_test = adjusted_r2_score(r2_test, len(ground_truth_labels), num_features)

print(f"Adjusted R2 Score on Test Set: {adjusted_r2_test}")

R2 Score on Test Set: 0.9867966404000411


## 4) 추론

In [ ]:
# Function to preprocess the new sentence and get model prediction
def predict_polarity(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

     # Move the input tensors to the specified device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass through the model
    with torch.no_grad():
        logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    # Convert logits to predicted polarity
    polarity = logits.item()
    return polarity

# Example usage:
while True:
    # Get input sentence from the user
    input_sentence = input("리뷰를 입력하세요. :")

    # Quit loop if 'q' is entered
    if input_sentence.lower() == 'q':
        break

    # Get the polarity prediction
    polarity_prediction = predict_polarity(input_sentence)

    # Display the result
    print(f"입력된 리뷰의 감성점수입니다.: {polarity_prediction:.2f}")

리뷰를 입력하세요. :술이 참 달고 맛있네요, 재구매 의사 있습니다!
입력된 리뷰의 감성점수입니다.: 1.32
리뷰를 입력하세요. :맛이 없어요, 쓴맛이 너무 심하네요.... 기대했는데 생각보다 별로여서 실망입니다.
입력된 리뷰의 감성점수입니다.: -9.43
리뷰를 입력하세요. :택배 포장이 부실해서 술이 다 쏟아졌어요. 반품해야겠네요.
입력된 리뷰의 감성점수입니다.: -3.19
리뷰를 입력하세요. :제가 지금까지 먹은 술 중 최고입니다! 이제부터 제 최애는 ㅇㅇㅇ이에요! 좋은 술 감사합니다^^
입력된 리뷰의 감성점수입니다.: 5.05
리뷰를 입력하세요. :q
